In [ ]:
from ollama import chat
from pydantic import BaseModel, Field
import arcpy

In [ ]:
# Get the current ArcGIS Pro project and active map
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_view = aprx.activeMap

In [ ]:
# Access current ArcGIS Pro project and map
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_view = aprx.activeMap

# Print all layer names
for lyr in map_view.listLayers():
    print(lyr.name)

In [ ]:
class LayerWhere(BaseModel):
    layer: str = Field(
        ...,
        description="The name of the layer.",
    )
    where: str = Field(
        ..., 
        description="An SQL expression used to select a subset of records."
    )
    

In [ ]:
instructions = [
    "You are an expert at converting a user question to a structured JSON output.", 
    "The JSON output consists of two properties:",
    " - layer: Thiss will contain the name of the layer or the table to query.",
    " - where: This will contain a SQL expression that will be used to select a subset of records from the layer.",
    "The following is the metadata for the layers",
    "Use the layer named Events By Magnitude to analyze earthquakes",
    "Layer Events By Magnitude: represents earthquakes",
    " - Field: mag (Float) – Earthquake magnitude",
    " - Field: place (String) – Location description",
    " - Field: time (Date) – Event timestamp",
    "Use this schema to understand how to build SQL queries against these layers.",
]


In [ ]:
messages = [
    {
        "role": "system", 
        "content": "\n".join(instructions),
    },
    {
        "role": "user", 
        "content" : "Show earthquakes with magnitude greater than 5.0",
    },
]

In [ ]:
response = chat(
    model = "llama3.2:latest",
    messages=messages, 
    format = LayerWhere.model_json_schema(),
    options ={
        "temperature": 0,
    },
)

layer_where = LayerWhere.model_validate_json(response.message.content)
print(layer_where)

In [ ]:
layer_name = layer_where.layer  # <-- This line defines the variable


In [ ]:
layer = next((lyr for lyr in map_view.listLayers() if lyr.name == layer_name), None)

if not layer:
    raise ValueError(f"Layer '{layer_name}' not found in the current map.")

In [ ]:
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=layer_where.layer,
    selection_type="NEW_SELECTION",
    where_clause=layer_where.where
)
